<h1 style="text-align:center;">Permutationally Invariant Network for Enhanced Sampling (PINES)</h1>

# Introduction
Permutationally Invariant Network for Enhanced Sampling (PINES) is a data-driven method for the automatic discovery of high variance collective variables (CVs) and enhanced sampling to obtain free energy landscapes of both single and multi-component molecular systems. It can be conceived as a permutationally invariant extension to the Molecular Enhanced Sampling with Autoencoders (MESA) method. PINES comprises three main components (Fig. 1):

1. Permutation Invariant Vector (PIV) featurization (F):  Translationally, rotationally, and permutationally invariant representation of the molecular system.
2. Universal function approximators (Autoencoders): Discovery of high-variance CVs {($\xi_1$, $\xi_2$, ...)}
3. Molecular dynamics (MD) simulations with CV biasing method: An enhanced sampling MD simulation of the system by biasing along the high variance CVs learned by the autoencoders to improve sampling of the system. We use Parallel-Bias Metadynamics (PBMetaD) in this tutorial, but any CV-enhanced sampling technique can be employed for this part.


<figure align="center">
  <img src="./notebook_src/pines1.png" alt="PINES overview">
  <figcaption>Fig. 1:  Overview of PINES for automatic discovery of high-variance CVs and biasing the system using CV-enhanced sampling to overcome the sampling challenges of MD simulations in determining free energy landscapes of both single and multi-molecular systems. (a) We perform MD simulations to sample the phase space of the system. The snapshot shows a representation of a NaCl system configuration in water with Na, Cl, O, and H colored in yellow, green, red, and white, respectively. (b) Representation of the system configurations sampled during MD simulation in Cartesian space, where {x<sub>i</sub>, y<sub>i</sub>, z<sub>i</sub>} represent the x, y, and z positions of atom i in the system. (c) Conversion of the system configurations sampled during MD simulation to a permutationally, rotationally, and translationally invariant representation using PIVs. (d) Learning differentiable and high-variance nonlinear CVs using autoencoders. We use a feed-forward network as a universal function approximator for both the encoder and decoder parts of the autoencoders. (e) Principal component analysis to align the manifold discovered using autoencoders with the highest variance directions. (f) CV enhanced sampling using PBMetaD or termination of PINES upon the convergence of discovered CVs.
</figcaption>
</figure>

In this introductory tutorial, we apply PINES to study the association or dissociation process of a NaCl ion pair in water. This is a classic system, where it is well known that the rearrangement of water molecules in the solvation shell of ions plays a critical role in driving the association or dissociation of NaCl. In other words, enhanced sampling of association or dissociation of Na+ and Cl- ions in water may require explicit biasing along water degrees of freedom. In this tutorial, we show how the permutational invariance built into PINES helps us discover solvent-inclusive high-variance CVs. This is challenging using traditional molecular representational techniques because water molecules are indistinguishable, and therefore their identity should not be embedded in the manifold of the system or the CVs. PINES offers a computationally efficient solution to handle this problem using PIV representation that automatically removes translational, rotational, and permutational variances. Below, we walk through each step of applying PINES to discover high-variance CVs and perform enhanced sampling MD simulations to study the free energy landscape of NaCl association or dissociation in water.

## Tutorial data directory structure

The tutorial's data directory is setup at follows:

* `0.Unbiased`: This folder contains the seeding simulations needed for training the autoencoder and tuning the switching function.
    * `1_solvate`: Creating the simulation box with 1 NaCl ion solvated in TIP3P water. 
    * `2_em`: Minimizing the simulation box to relax non-physical structures followed by a short NVT simulation for equilibration.
    * `3_md`: An unbiased MD run to seed the autoencoder training.
* `1.NN`
    * Training the autoencoder using the seeding unbiased simulation data or the biased data from the previous round to find high-variance CVs.
* `2.Biased`
    * Biased MD run with PBMetaD and the CVs discovered by PINES.
* `3.Check_Convergence`
    * Check whether the PINES CVs have converged by looking and the latest space dimentionality and cosine similarity of the discovered CVs.

## Importing neccessary packages

In [1]:
import os
import subprocess
from pathlib import Path
import pandas as pd
import numpy as np

A simple helper class for changing directories

In [ ]:
class cd:
    """Context manager for changing the current working directory"""
    def __init__(self, newPath):
        self.newPath = Path(newPath)
    def __enter__(self):
        self.savedPath = Path.cwd()
        try:
            os.chdir(self.newPath)
        except:
            raise ValueError("Path does not exist.")
    def __exit__(self, etype, value, traceback):
        os.chdir(self.savedPath)

## Setting up path variables

In [ ]:
PLUMED_EXEC = Path("")
GMX_EXEC = Path("")
BASE_DIR = Path("") # where this notebook is

Making sure we start from `BASE_DIR`

In [ ]:
os.chdir(BASE_DIR)

# Seeding PINES: Unbiased simulation

Before we can start biasing for enhanced sampling, we must train our autoencoder with a short trajectory of our unbiased system. We will solvate a NaCl ion pair in TIP3P water molecules in a cubic box with an edge length of 2.5 nm. We then minimize our simulation box and run a 100 ps NVT equilibration run. We will then use the equilibrated box for a 50 ns NVT production run, which we will use to train our autoencoder.

## 1_solvate
We will use GROMACS's built-in `solvate` program to create a box of water.

In [ ]:
SOLVATE_DIR = BASE_DIR/"0.Unbiased"/"1_solvate"

In [ ]:
subprocess.run(f"{GMX_EXEC} solvate -cs spc216.gro -box 2.5 -o {SOLVATE_DIR/"box.gro"} -p {SOLVATE_DIR/"topol.top"}")
subprocess.run(f"{GMX_EXEC} pdb2gmx -f {SOLVATE_DIR/"box.gro"} -o {SOLVATE_DIR/"box.gro"} -water tip3p")

Add a single pair of $\text{Na}^+$ and $\text{Cl}^+$ ions to the water box with `genion`

In [ ]:
subprocess.run(f"{GMX_EXEC} grompp -f {SOLVATE_DIR/"ions.mdp"} -c {SOLVATE_DIR/"box.gro"} -p {SOLVATE_DIR/"topol.top"} -o {SOLVATE_DIR/"ions.tpr"} -maxwarn 2")
subprocess.run(f"echo SOL | {GMX_EXEC} genion -s {SOLVATE_DIR/"ions.tpr"} -o {SOLVATE_DIR/"ions.gro"} -p {SOLVATE_DIR/"topol.top"} -pname NA -nname CL -np 1 -nn 1")

## 2_em

Minimize the structure of the simulation box we just created to relax non-physical structures.

In [ ]:
EM_DIR = BASE_DIR/"0.Unbiased"/"2_em"

In [ ]:
subprocess.run(f"cp {SOLVATE_DIR/"ions.gro"} {SOLVATE_DIR/"topol.top"} {EM_DIR/"."}")
subprocess.run(f"{GMX_EXEC} grompp -f {EM_DIR/"em.mdp"} -c {EM_DIR/"ions.gro"} -p {EM_DIR/"topol.top"} -o {EM_DIR/"em.tpr"} -maxwarn 3")
with cd(f"{EM_DIR}"):
    subprocess.run(f"{GMX_EXEC} mdrun -deffnm em")

100 ps NVT equilibration run

In [ ]:
subprocess.run(f"{GMX_EXEC} grompp -f {EM_DIR/"nvt.mdp"} -c {EM_DIR/"em.gro"} -p {EM_DIR/"topol.top"} -o {EM_DIR/"nvt.tpr"} -maxwarn 3")
with cd(f"{EM_DIR}"):
    subprocess.run(f"{GMX_EXEC} mdrun -deffnm nvt")

## 2_md

50 ns NVT production run for seeding the autoencoder

In [ ]:
MD_DIR = BASE_DIR/"0.Unbiased"/"3_md"

In [ ]:
subprocess.run(f"cp {EM_DIR/"nvt.gro"} {EM_DIR/"nvt.cpt"} {EM_DIR/"topol.top"} {MD_DIR/"."}")
subprocess.run(f"{GMX_EXEC} grompp -f {MD_DIR/"md.mdp"} -c {MD_DIR/"nvt.gro"} -t {MD_DIR/"nvt.cpt"} -p {MD_DIR/"topol.top"} -o {MD_DIR/"md.tpr"} -maxwarn 2")
with cd(f"{MD_DIR}"):
    subprocess.run(f"{GMX_EXEC} mdrun -deffnm md")

# 1. Calculating PIVs

The PIV featurization is constructed from pairwise distances and is naturally invariant to rigid translation and rotation. To make PIVs permutationally invariant, they are constructed in three steps:

1. We calculate an adjacency matrix populated with the Euclidean pairwise distances $a_{ij}$.  The adjacency matrix is then ordered into a block structure where each block contains the pairwise distances of a pair of atom types. For example, for our system with O, H, Na, and Cl atoms, the adjacency matrix can have up to 10 blocks: O-O, O-H, O-Cl, O-Na, H-H, H-Cl, H-Na, Na-Na, Na-Cl, Cl-Cl. Because our goal is to study the Na-Cl ion pair, we will only focus on the groups involving the ion: Na-Cl, Na-O, Na-H, Cl-O, and Cl-H.

2. We pass the adjacency matrix through a rational switching function
$$V_{ij} = \frac{1-\left ( \frac{a_{ij} - d_0}{r_0} \right )^n}{1-\left ( \frac{a_{ij} - d_0}{r_0} \right )^m}$$

where $d_0$ is the minimum interparticle distance, $r_0$ is the interparticle distance, which is the midpoint of the switching function, and $n$ and $m$ are non-negative numbers controlling the steepness of the switching function. The switching function scales the distances to the (0, 1) interval, where values closer to 1 indicate close pairs, 0 for pairs that are far away, and a switching range that can be controlled to tuned sensitivity to $a_{ij}$ values. These parameters are typically selected from an RDF collected over a short, unbiased trajectory. $r_0$ is chosen to be the mean of the first and second peaks of the solvation shell from the RDF. $n$ is tuned so that the $v_{ij}$ value for the pairwise distance at the first peak is close to 0.9, and the $v_{ij}$ value for the pairwise distance at the second peak is close to 0.1. For our system, $d_0$ is set to 0 and $m=2n$.

3. We sort $v_{ij}$ s in each block of the switched adjacency matrix and concatenate them into a single vector to produce the PIV. This step is the critical step in forming a permutationally invariant vector.

<figure align="center">
  <img src="./notebook_src/pines2.png" alt="PIV calculation">
  <figcaption>Fig. 2:  Overview of the PIV calculation procedure. (a) The MD trajectory is used to calculate the adjacency matrix. (b) We calculate the adjacency matrix for all atoms and order in blocks of atom type pairs. (c) We pass the adjacency matrix through a switching function to get values between (0, 1) and highlight the importance of the closest solvation shells. (d) The switching function values are vectorized and sorted per atom type block to enforce permutational invariance.
</figcaption>
</figure>

## 1.1 PLUMED input file

We use the `PIV` module in `PLUMED` to calculate the PIV values

In [ ]:
PIV_DIR = BASE_DIR/"0.Unbiased"/"4_calcPIV"

In [ ]:
PLUMED_PIV = """
PIV ...
LABEL=piv
PRECISION=10000
NLIST
ONLYCROSS
REF_FILE=nvt.pdb
PIVATOMS=5
ATOMTYPES=NA,CL,OW,HW1,HW2
SFACTOR=1.0,1.0,1.0,1.0,1.0
SORT=1.0,1.0,1.0,1.0,1.0
SWITCH1={RATIONAL R_0=0.38 MM=12 NN=6}
SWITCH2={RATIONAL R_0=0.35 MM=12 NN=6}
SWITCH3={RATIONAL R_0=0.41 MM=12 NN=6}
SWITCH4={RATIONAL R_0=0.40 MM=12 NN=6}
SWITCH5={RATIONAL R_0=0.28 MM=12 NN=6}
NL_CUTOFF=5.0,1.5,1.5,1.5,1.5
NL_STRIDE=1.0,1.0,1.0,1.0,1.0
NL_SKIN=0.1,0.1,0.1,0.1,0.1
PIVREP
NL_CONSTANT_SIZE=15
WRITEPIVTRAJ
WRITEPIVSTRIDE=100000000
WRITEANNSTRIDE=100000000
... PIV

PRINT ARG=piv.* FILE=colvar STRIDE=1
"""

# DESCRIBE THE PARAMETERS AND REPLACE SOME OF THEM WITH `__FILL__` TO MAKE IT AN EXCERCISE

## 1.2 Run PLUMED to get PIVs

In [ ]:
# call plumed driver on the trajectory to get the PIVs
subprocess.run(f"{PLUMED_EXEC} driver --plumed plumed_calcPIV.dat --mf_xtc {MD_DIR/"md.xtc"} >& out.out")

Read in the PLUMED outputs, clean up, and save as a numpy array

In [18]:
# load the colvar file as a pandas dataframe
PIVs = pd.read_csv(f"{PIV_DIR}/out.out", sep='\s+', header=None, skiprows=1)
# drop the time column
PIVs.drop([0], axis=1, inplace=True)
# view the result
PIVs.head(5)

,1,2,3,4,5,6,7,8,9,10,...,82,83,84,85,86,87,88,89,90,91
0,0.0037,0.129213,0.130113,0.146315,0.151515,0.207921,0.260126,0.267627,0.314331,0.329133,...,0.245825,0.257626,0.310131,0.327633,0.633063,0.647765,0.701370,0.799180,0.831083,0.867087
1,0.0014,0.174917,0.175718,0.212721,0.212921,0.216122,0.261826,0.265627,0.299430,0.479148,...,0.261326,0.261326,0.415842,0.533753,0.660166,0.741174,0.750075,0.782878,0.832083,0.844984
2,0.0011,0.141614,0.156616,0.159616,0.184018,0.251025,0.255626,0.263726,0.267027,0.270827,...,0.178718,0.320332,0.417242,0.455646,0.670067,0.684468,0.686969,0.711871,0.764876,0.790679
3,0.0007,0.131913,0.147815,0.179918,0.194819,0.207821,0.227323,0.256126,0.265127,0.301130,...,0.278128,0.294129,0.346835,0.365537,0.706871,0.743074,0.816182,0.852185,0.864786,0.865387
4,0.0008,0.162216,0.164016,0.174717,0.192619,0.196420,0.222322,0.229223,0.362136,0.378238,...,0.234123,0.255426,0.326333,0.626963,0.766377,0.781078,0.819882,0.821982,0.860986,0.869487


Save the pandas dataframe as a numpy matrix

In [ ]:
np.save("./PIVs.npy", PIVs.to_numpy(float))

# 2. Training the neural network on the initial data

## 2.1 load the model

In [ ]:
pass

## 2.2 Pass the calculated PIVs as training data

In [ ]:
pass

## 2.3 Model design and validation

[include elbow plots for the number of hiddens nodes]

## 2.4 Output model weights for PLUMED

[describe what's happening]

In [ ]:
pass

# 3. Bias the simulation with PINES in PLUMED

[Describe what we are doing]

## 3.1 load the model weights

In [19]:
# load the wieghts and create a string to be replaced in the PLUMED input
pass # DO STUFF TO GET THE WEIGHTS
ANN_WEIGHTS=f"""
WEIGHTS0={}
WEIGHTS1={}
WEIGHTS2={}
WEIGHTS3={}
WEIGHTS4={}
BIASES0={}
BIASES1={}
BIASES2={}
BIASES3={}
BIASES4={}
GAMMAS0={}
GAMMAS1={}
GAMMAS2={}
GAMMAS3={}
BETAS0={}
BETAS1={}
BETAS2={}
BETAS3={}
EXPECTATIONS0={}
EXPECTATIONS1={}
EXPECTATIONS2={}
EXPECTATIONS3={}
VARIANCES0={}
VARIANCES1={}
VARIANCES2={}
VARIANCES3={}
"""

## 3.2 PINES PLUMED input file

[Describe what's happening]

In [ ]:
PLUMED_PINES = f"""
PINES ...
LABEL=pines
PRECISION=10000
NLIST
ONLYCROSS
REF_FILE=md.pdb
PINESATOMS=5HGGF_NS  ar
ATOMTYPES=NA,CL,OW,HW1,HW2
SFACTOR=1.0,1.0,1.0,1.0,1.0
SORT=1.0,1.0,1.0,1.0,1.0
SWITCH1={{RATIONAL R_0=0.38 MM=12 NN=6}}
SWITCH2={{RATIONAL R_0=0.35 MM=12 NN=6}}
SWITCH3={{RATIONAL R_0=0.41 MM=12 NN=6}}
SWITCH4={{RATIONAL R_0=0.40 MM=12 NN=6}}
SWITCH5={{RATIONAL R_0=0.28 MM=12 NN=6}}
NL_CUTOFF=5.0,1.5,1.5,1.5,1.5
NL_STRIDE=10.0,10.0,10.0,10.0,10.0
NL_SKIN=0.1,0.1,0.1,0.1,0.1
NL_CONSTANT_SIZE=10
... PINES

ANN ...
LABEL=ann
ARG=pines.*
NUM_LAYERS=6
NUM_NODES=61,64,32,16,3,3
ACTIVATIONS=BNTanh,BNTanh,BNTanh,BNTanh,Linear
{ANN_WEIGHTS}
APPLY_BATCH_NORM
... ANN


PBMETAD ...
ARG=ann.node-0,ann.node-1,ann.node-2 SIGMA=0.1,0.1,0.1 HEIGHT=1.2
TEMP=300 PACE=500 BIASFACTOR=2 LABEL=pb
FILE=hills_pc0.out,hills_pc2.out,hills_pc3.out
GRID_MIN=-2.5,-2.5,-2.5 GRID_MAX=2.5,2.5,2.5 GRID_BIN=5000,5000,5000
... PBMETAD

# monitor the cv and the metadynamics bias potential
PRINT STRIDE=1 ARG=ann.node-0,ann.node-1,ann.node-2,pb.bias FILE=colvar.out

"""

Save the plumed file

In [ ]:
with open("./plumed_PINES_input.dat", "w") as f:
    f.write(PLUMED_PINES)

## 3.3 Run the biased simulation

[Describe what's happening]

In [ ]:
subprocess.run(f"{GMX_EXEC} mdrun -s md_50ns.tpr -deffnm biased -plumed plumed_PINES_input.dat -cpi biased.cpt")

# 4. Processing the results